В этом задании начнем с простого изучения данных.

Необходимо посмотреть на данные и разобраться с ними: 
1. Выявить порядковые категориальные данные.
2. Посмотреть на сатистики по данным.
3. Удостовериться, что данные не противоречат логике.

 Как делать первый пункт:
     * Описать, как ты понимаешь каждую колонку. Колонки с текстами, картинками и id сейчас не рассматриваем. 
     * Доказать, что колонка является просто категориальной или порядковой категориальной.
     * Просто категориальные оставляем, а порядковые проверяем на логичность нумерации. В случае нелогичности перенумеровываем.
     * Как проверяем порядковые. Смотрим по логике, напрмер, рассматриваем, как соотносятся размеры пород с с размерами в колонке размеров.
     
Как делать второй пункт:
    * Посмотреть и визуализировать, например, сколько каких животных по возрасту.
    * Или сколько каким типам животных нужно для того, чтобы из забрали.
    
Третий пункт вытекает из первых двух, обобщение собранной информации. 


### Примеры кода, который может быть полезен

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import os
import json
from pandas.io.json import json_normalize
from pprint import pprint
from PIL import Image
from IPython.display import display, HTML

In [ ]:
breeds = pd.read_csv('../input/breed_labels.csv')
colors = pd.read_csv('../input/color_labels.csv')
states = pd.read_csv('../input/state_labels.csv')
train = pd.read_csv('../input/train/train.csv')
test = pd.read_csv('../input/test/test.csv')

train.head()

**Порядковые и категориальные данные**

**Type** - Вид животного: кошка/собака. Категориальный

**Age** - Возраст. Порядковый категориальный признак.

**Breed1** - Порода, категориальный признак.

**Breed2** - Вторая порода (для метисов). Категориальный признак.

**Gender** - Пол животного, категориальный.

**Color1,2,3** - Цвета, которые встречаются в окрасе животного. Категориальный признак.

**MaturitySize** - Размер животного, порядковый категориальный признак.

**FurLength** - Длина шерсти, порядковый категориальный.

**Vaccinated** - Вакцинировано ли животное (да/нет/не известно). Категориальный признак.

**Dewormed** - Обработано ли животное от глистов. Категориальный признак.

**Sterilized** - Стерилизовано ли животное. Категориальный.

**Health** - Некий показатель здоровья, думаю, что тоже категориальный.

**Quantity** - Количество животных, которое пристраивают. Порядковый.

**Fee** - Стоимость, порядковый.

**State** - В каком штате находится животное. Категориальный.

**RescuerID** - ID человека (или организации), который пристраивает животное. Категориальный.

In [ ]:
sns.distplot(train['AdoptionSpeed'], bins = 5, kde=False)

**Смотрим, как распределены порядковые данные**

In [ ]:
sns.distplot(train['Age'], bins = 100, kde=False)

In [ ]:
train.Age.unique()

В датасете встречаются весьма древние животные. Но, возможно, в некоторых случаях указано не количество лет, а количество дней? Попробую понять по фотографии, права ли я.

In [ ]:
train[train['Age'] == 81]['PetID']

In [ ]:
from PIL import Image
from io import BytesIO
import requests
image = Image.open("../input/train_images/2ba708d92-1.jpg")
image

Милаха <3 Но, похоже, это взрослое животное. Тогда я буду считать ошибкой возраст больше 20. Удалю его и заменю средним значением

In [ ]:
train['Age'] = train['Age'].fillna( method='pad')

In [ ]:
train.loc[train['Age'] > 20, 'Age'] = train['Age'].mean()

In [ ]:
train['Age']

In [ ]:
sns.distplot(train['Age'], bins = 20, kde=False)

**Переходим к породам**

In [ ]:
breed_dict = dict(zip(breeds['BreedID'], breeds['BreedName']))

In [ ]:
animal_dict = {1: 'Dog', 2: 'Cat'}

In [ ]:
train.head()

In [ ]:
def make_features(train):
    train['Named'] = (~train['Name'].isnull() | train['Name'].str.contains('(No | Puppies | Kitty | Puppy | Kitten)')).astype(int)
    train['PureBreed'] = (train.Breed2==0 | ~train.Breed1.isin(breeds[breeds.BreedName.str.contains('Domestic' or 'Mixed')].BreedID.values)).astype(int)
    train = train.drop('Breed2', axis=1)

    train = train.select_dtypes(exclude=[object])
    return train

dealed_train = make_features(train)

In [ ]:
dealed_train['Type'] = dealed_train['Type'].map(animal_dict)
dealed_train['Breed1_new'] = dealed_train['Breed1'].map(breed_dict)

In [ ]:
train_dogs = dealed_train[dealed_train['Type'] == 'Dog']
train_cats = dealed_train[dealed_train['Type'] == 'Cat']

In [ ]:
train_dogs.Breed1_new.unique()

In [ ]:
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 4))
sns.countplot(data=train_dogs, x='PureBreed',hue='AdoptionSpeed', ax=ax1)
plt.title('Dogs adoption speed')
sns.countplot(data=train_cats, x='PureBreed',hue='AdoptionSpeed', ax=ax2)
plt.title('Cats adoption speed')

*И у меня сразу два вопроса: как мне написать, чтобы чистопородными не считались животные, в названии породы у которых есть "Mixed"? Почему-то мой код его не видит ((
И как сделать, чтобы названия отображались под обоими графиками?*

In [ ]:
dealed_train.head()

*Не понимаю, куда делась колонка 'PetID', если я ее не удаляла -___-*

In [ ]:
train['Breed1_new'] = train['Breed1'].map(breed_dict)
train['Type'] = train['Type'].map(animal_dict)

In [ ]:
train.groupby(['Type', 'Breed1_new', 'FurLength'])['PetID'].count()

*Понять не могу, что можно сделать с этим (( Где посмотреть PetID выбросов?*

In [ ]:
train_dogs['Breed1_new'].value_counts().iloc[:5]

In [ ]:
train_cats['Breed1_new'].value_counts().iloc[:5]

Возможно, животных, у которых есть имена, забирают быстрее?

In [ ]:
plt.figure(figsize=(14, 6));
sns.factorplot(x='Named', y='AdoptionSpeed', data=dealed_train, kind='bar')
plt.title('Having name is good or bad?')

Вероятно, люди предпочитают сами придумывать имена своим новым питомцам

In [ ]:
plt.figure(figsize=(28, 12));
sns.barplot(x='PhotoAmt', y='AdoptionSpeed', data=train)
plt.title('What about photos?')

А вот 5-10 фото не помешают

In [ ]:
sns.lmplot(x='Fee',y='AdoptionSpeed',data=train,hue='Type')

*Ого, какая странная зависимость. Такое ощущение, что она какая-то искусственная. Посмотрю на нее пристальнее завтра :)*

1. ### Еще немного важного, что можно изучать:
    * Метрики: немного про них тут https://github.com/esokolov/ml-course-hse/blob/master/2017-fall/lecture-notes/lecture02-linregr.pdf
    Можно использовать, как источник ключевых слов.
    * Градиентный бустинг: https://habr.com/en/company/ods/blog/327250/